# Train

In [ ]:
import random
import os
import math
import numpy as np
import pandas as pd
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, BatchSampler
from torchinfo import summary
from torch import optim

from fastai.vision.all import DataLoaders, Learner, CSVLogger
from fastai.optimizer import OptimWrapper

from torchscale.architecture.config import EncoderConfig
from torchscale.architecture.encoder import Encoder

## §0 `utils.py` def `seed_everything`

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

## §1 `dataset.py`

### §1.1 def `prepare_sensors`

In [ ]:
def prepare_sensors():
    sensors = pd.read_csv("data/sensor_geometry.csv").astype(
        {
            "sensor_id": np.int16,
            "x": np.float32,
            "y": np.float32,
            "z": np.float32,
        }
    )
    sensors["string"] = 0
    sensors["qe"] = 0  # or 1, Quantum Efficiency

    for i in range(len(sensors) // 60):
        start, end = i * 60, (i * 60) + 60
        sensors.loc[start:end, "string"] = i# classify sensors with string

        # High Quantum Efficiency in the lower 50 DOMs - https://arxiv.org/pdf/2209.03042.pdf (Figure 1)
        if i in range(78, 86):
            start_veto, end_veto = i * 60, (i * 60) + 10
            start_core, end_core = end_veto + 1, (i * 60) + 60
            sensors.loc[start_core:end_core, "qe"] = 1  # 1.35

    # https://github.com/graphnet-team/graphnet/blob/b2bad25528652587ab0cdb7cf2335ee254cfa2db/src/graphnet/models/detector/icecube.py#L33-L41
    # Assume that "rde" (relative dom efficiency) is equivalent to QE
    sensors["x"] /= 500
    sensors["y"] /= 500
    sensors["z"] /= 500

    return sensors

### §1.2 class `IceCubeDataset`

In [ ]:
class IceCubeDataset(Dataset):
    def __init__(self, dim_base=128, batch_id=1):
        # dim_base for nomalize
        self.dim_base = dim_base
        
        # 1. load `batch_1.parquet` and concat the same event_id together
        batch_1 = pd.read_parquet(os.path.join("data/train", "batch_"+str(batch_id)+".parquet"))# data/train/batch_1.parquet
        batch_1 = batch_1.groupby('event_id').agg({
            'sensor_id': list,
            'time': list,
            'charge': list,
            'auxiliary': list
        })
        batch_1.reset_index(drop=True, inplace=True)# event_id -> index
        self.batch_1 = batch_1
        
        # 2. geometry
        sensors = prepare_sensors()
        self.geometry = torch.from_numpy(sensors[["x", "y", "z"]].values.astype(np.float32))
        self.qe = sensors["qe"].values
        
        # 3. load `train_meta.parquet` to get target: azimuth and zenith
        train_meta = pd.read_parquet('data/train_meta.parquet')
        batch_1_meta = train_meta[train_meta['batch_id'] == batch_id]
        batch_1_meta.reset_index(drop=True, inplace=True)# event_id -> index
        self.batch_1_meta = batch_1_meta
        
        assert len(self.batch_1)==len(self.batch_1_meta), "batch_1 and batch_1_meta length doesn't match"

    def __len__(self):
        return len(self.batch_1)
    
    def __getitem__(self, index):
        # 1. for each event in `batch_1.parquet`
        batch_1_event = self.batch_1.loc[index]
        sensor_id = np.array(batch_1_event["sensor_id"])
        time = np.array(batch_1_event["time"])
        time = (time - 1e4) / 3e4
        charge = np.array(batch_1_event["charge"])
        charge = np.log10(charge) / 3.0
        auxiliary = np.array(batch_1_event["auxiliary"])

        # normalize so that the length of each event is 128
        dim_base = len(sensor_id)
        # the output of this method
        dim_base_0 = dim_base
        if dim_base < self.dim_base:
            # assignment
            sensor_id = np.pad(sensor_id, (0, max(0, self.dim_base - dim_base)))
            time = np.pad(time, (0, max(0, self.dim_base - dim_base)))
            charge = np.pad(charge, (0, max(0, self.dim_base - dim_base)))
            auxiliary = np.pad(auxiliary, (0, max(0, self.dim_base - dim_base)))
        else:
            # generate a randomly arranged index array ids
            ids = torch.randperm(dim_base).numpy()
            # negtive index
            auxiliary_n = np.where(~auxiliary)[0]
            # positive index
            auxiliary_p = np.where(auxiliary)[0]
            # choose as many as ids in the negtive index
            ids_n = ids[auxiliary_n][: min(self.dim_base, len(auxiliary_n))]
            # the rest is positive index
            ids_p = ids[auxiliary_p][: min(self.dim_base - len(ids_n), len(auxiliary_p))]
            # concat into a new index
            ids = np.concatenate([ids_n, ids_p])
            # sort the index (from small to big)
            ids.sort()
            # assignment
            sensor_id = sensor_id[ids]
            time = time[ids]
            charge = charge[ids]
            auxiliary = auxiliary[ids]
            dim_base = len(ids)

        # 2. geometry
        sensor_id = torch.from_numpy(sensor_id).long()
        pos = self.geometry[sensor_id]
        pos[dim_base:] = 0
        qe = self.qe[sensor_id]
        qe[dim_base:] = 0
        
        # 3. target
        azimuth = self.batch_1_meta.loc[index].azimuth
        zenith = self.batch_1_meta.loc[index].zenith
        target = np.array([azimuth, zenith]).astype(np.float32)
        target = torch.from_numpy(target)
        
        # 4. mask
        mask = torch.zeros(self.dim_base, dtype=torch.bool)
        mask[:dim_base] = True

        return {
            "time": torch.from_numpy(time).float(),
            "charge": torch.from_numpy(charge).float(),
            "auxiliary": torch.from_numpy(auxiliary).long(),
            "pos": pos,
            "mask": mask,
            "qe": qe,
            "dim_base_0": dim_base_0
        }, {"target": target}

## §2 `model.py`

### §2.1 class `SinusoidalPosEmb`

In [ ]:
class SinusoidalPosEmb(nn.Module):# Sinusoidal Position Embedding
    def __init__(self, dim=16, M=10000):
        super().__init__()
        self.dim = dim
        self.M = M

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(self.M) / half_dim
        emb = torch.exp(torch.arange(half_dim, device=device) * (-emb))
        emb = x[..., None] * emb[None, ...]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

### §2.2 class `IceCubeEmbedding`

- `dim_base` is the dimension of normalize, see `IceCubeDataset`, `dim` is the dimension of embedding

In [ ]:
class IceCubeEmbedding(nn.Module):
    def __init__(self, dim=384, dim_base=128):
        super().__init__()
        self.emb = SinusoidalPosEmb(dim=dim_base)
        self.aux_emb = nn.Embedding(2, dim_base // 2)
        self.emb2 = SinusoidalPosEmb(dim=dim_base // 2)
        self.proj = nn.Sequential(
            nn.Linear(6 * dim_base, 6 * dim_base),
            nn.LayerNorm(6 * dim_base),
            nn.GELU(),
            nn.Linear(6 * dim_base, dim),
        )

    def forward(self, x, Lmax=None):
        pos = x["pos"] if Lmax is None else x["pos"][:, :Lmax]
        charge = x["charge"] if Lmax is None else x["charge"][:, :Lmax]
        time = x["time"] if Lmax is None else x["time"][:, :Lmax]
        auxiliary = x["auxiliary"] if Lmax is None else x["auxiliary"][:, :Lmax]
        length = torch.log10(x["dim_base_0"].to(dtype=pos.dtype))

        x = torch.cat(
            [
                #self.cls_token,
                self.emb(4096 * pos).flatten(-2),
                self.emb(1024 * charge),
                self.emb(4096 * time),
                self.aux_emb(auxiliary),
                self.emb2(length).unsqueeze(1).expand(-1, pos.shape[1], -1)
            ],
            -1,
        )
        x = self.proj(x)
        return x

### §2.3 class `IceCubeModel`

[microsoft/torchscale (github.com)](https://github.com/microsoft/torchscale/tree/main)

1. Use `Encoder` from `torchscale`
    
    ```python
    from torchscale.architecture.config import EncoderConfig
    from torchscale.architecture.encoder import Encoder

    encoder_config = EncoderConfig(encoder_embed_dim = 384)
    encoder = Encoder(encoder_config)

    # to test encoder:
    # https://github.com/microsoft/torchscale/blob/main/tests/test_encoder.py
    dummy = torch.rand(32, 129, 384)
    dummy = encoder(src_tokens=None, token_embeddings=dummy)
    dummy = dummy['encoder_out']
    print(dummy.shape)
    ```
    
    The output is:
    
    ```python
    torch.Size([32, 129, 384])
    ```

2. First take the slices then use `self.proj_out` to get xyz. 32 is batch_size
    
    ```python
    x = torch.randn(32, 129, 384)
    print(x[:, 0, :].shape)
    proj_out = nn.Linear(384, 3)
    x = proj_out(x[:, 0, :])
    print(x.shape)
    ```
    
    The output is:
    
    ```Shell
    torch.Size([32, 384])
    torch.Size([32, 3])
    ```

In [ ]:
class IceCubeModel(nn.Module):
    def __init__(
        self,
        dim=384,
        dim_base=128
    ):
        super().__init__()
        self.icecube_embedding = IceCubeEmbedding(dim, dim_base)
        encoder_config_1 = EncoderConfig(
            encoder_attention_heads=6,
            encoder_embed_dim=dim,
            encoder_ffn_embed_dim=1536,# Feed Forward Network
            encoder_layers=4,
            # relative position bias, see [1910.10683]
            rel_pos_buckets=32,
            max_rel_pos=dim_base
        )
        self.encoder_1 = Encoder(encoder_config_1)
        self.cls_token = nn.Linear(dim, 1, bias=False)
        encoder_config_2 = EncoderConfig(
            encoder_attention_heads=12,
            encoder_embed_dim=dim,
            encoder_ffn_embed_dim=1536,# Feed Forward Network
            encoder_layers=12
        )
        self.encoder_2 = Encoder(encoder_config_2)
        self.proj_out = nn.Linear(dim, 3)

    def forward(self, x0):
        # mask
        mask = x0["mask"]
        Lmax = mask.sum(-1).max()
        # cls token
        batch_size, _ = mask.shape
        cls_token = self.cls_token.weight.unsqueeze(0).expand(batch_size, -1, -1)
        # x
        x = self.icecube_embedding(x0, Lmax)
        # encoder 1
        x = self.encoder_1(src_tokens=None, token_embeddings=x)
        x = x['encoder_out']
        # concat cls token
        x = torch.cat([cls_token, x], 1)
        # encoder 2
        x = self.encoder_2(src_tokens=None, token_embeddings=x)
        x = x['encoder_out']
        x = self.proj_out(x[:, 0, :])# get cls token, then xyz
        return x

In [ ]:
summary(IceCubeModel())

## §3 `loss.py`

### §3.1 def `mse_loss`

Shape of input:
- `pred`: shape is [batch_size, 3], "3" is xyz.
- `y["target"]`: shape is [batch_size, 2], "2" is azimuth and zenith.

In [ ]:
def mse_loss(pred, y):
    pred = pred.float()
    l = torch.norm(pred.float(), dim=-1).unsqueeze(-1)
    pred = pred.float()
    
    sa2 = torch.sin(y["target"][:, 0])
    ca2 = torch.cos(y["target"][:, 0])
    sz2 = torch.sin(y["target"][:, 1])
    cz2 = torch.cos(y["target"][:, 1])
    target = torch.stack([sa2 * sz2, ca2 * sz2, cz2], -1)
    
    loss = nn.MSELoss()(pred, target)
    return loss

### §3.2 def `competition_metric`

Shape of input:
- `pred`: shape is [batch_size, 3], "3" is xyz.
- `y["target"]`: shape is [batch_size, 2], "2" is azimuth and zenith.

Official metric of the kaggle competition：$$\Psi = \arccos (\sin{\vartheta_\text{true}}\sin{\vartheta_\text{reco}}(\cos{{\varphi_\text{ture}}}\cos{{\varphi_\text{reco}}}+\sin{{\varphi_\text{ture}}}\sin{{\varphi_\text{reco}}})+\cos{\vartheta_\text{true}}\cos{\vartheta_\text{reco}})$$

In [ ]:
def competition_metric(pred, y):
    pred = F.normalize(pred.double(), dim=-1)

    sa2 = torch.sin(y["target"][:, 0])#sin(azimuth)
    ca2 = torch.cos(y["target"][:, 0])
    sz2 = torch.sin(y["target"][:, 1])#sin(zenith)
    cz2 = torch.cos(y["target"][:, 1])

    scalar_prod = (
        pred[:, 0] * sa2 * sz2 + pred[:, 1] * ca2 * sz2 + pred[:, 2] * cz2
    ).clip(-1 + 1e-8, 1 - 1e-8)
    return torch.acos(scalar_prod).abs().mean(-1).float()

## §4 `train.py` def `train`

https://docs.fast.ai/examples/migrating_pytorch_verbose.html

In [ ]:
def train():
    seed_everything(42)
    
    # load data
    dataset = IceCubeDataset(dim_base=128, batch_id=1)
    data_size = len(dataset)
    train_size = int(0.8 * data_size)# 80% is train_loader
    indices = list(range(data_size))
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    train_batch_sampler = BatchSampler(
        torch.utils.data.SequentialSampler(train_indices),
        batch_size=32,
        drop_last=False
    )
    val_batch_sampler = BatchSampler(
        torch.utils.data.SequentialSampler(val_indices),
        batch_size=32,
        drop_last=False
    )
    train_loader = DataLoader(
        dataset,
        num_workers=4,
        batch_sampler=train_batch_sampler
    )
    val_loader = DataLoader(
        dataset,
        num_workers=1,
        batch_sampler=val_batch_sampler
    )
    dls = DataLoaders(train_loader, val_loader)
    model = IceCubeModel().cuda()
    
    # train
    learn = Learner(
        dls,
        model,
        path="BEiT3_29M_MSE_rel128",
        loss_func=mse_loss,
        metrics=competition_metric,
        opt_func=partial(OptimWrapper, opt=optim.Adam)
    )
    learn.fit_one_cycle(16, lr_max=5e-4, cbs=CSVLogger(fname='history.csv', append=True))
    learn.save("BEiT3_29M_MSE_rel128_0", with_opt=False)

In [ ]:
if __name__ == '__main__':
    train()